* `q2_k`: attention.vw 및 feed_forward.w2 텐서에는 Q4_K를 사용하고, 그 외 텐서에는 Q2_K를 사용합니다.
* `q3_k_l`: attention.wv, attention.wo, feed_forward.w2 텐서에는 Q5_K를 사용하고, 그 외 텐서에는 Q3_K를 사용합니다.
* `q3_k_m`: attention.wv, attention.wo, feed_forward.w2 텐서에는 Q4_K를 사용하고, 그 외에는 Q3_K를 사용합니다.
* `q3_k_s`: 모든 텐서에 Q3_K 사용
* `q4_0`: 기본 양자화 방식, 4비트.
* `q4_1`: q4_0보다 정확도가 높지만 q5_0만큼 높지는 않습니다.그러나 q5 모델보다 추론 속도가 빠릅니다.
* `q4_k_m`: attention.wv 및 feed_forward.w2 텐서의 절반에 Q6_K를 사용하고, 그 외에는 Q4_K ~를 사용합니다.
* `q4_k_s`: 모든 텐서에 Q4_K 사용
* `q5_0`: 정확도가 높고, 리소스 사용량이 많으며, 추론 속도가 느립니다.
* `q5_1`: 더 높은 정확도, 더 높은 리소스 사용량, 더 느린 추론.
* `q5_k_m`: attention.wv 및 feed_forward.w2 텐서의 절반에 Q6_K를 사용하고, 그 외에는 Q5_K를 사용합니다.~
* `q5_k_s`: 모든 텐서에서 Q5_K 사용
* `q6_k`: 모든 텐서에 Q6_K 사용
* `q8_0`: float16과 거의 구별할 수 없음.리소스 사용량이 많고 느립니다.대부분의 사용에는 권장되지 않습니다.

In [ ]:
MODEL_ID = "Redwood0/PiVoT-Merge-A-7B"
QUANTIZATION_METHODS = ["q5_k_m"]

MODEL_NAME = MODEL_ID.split('/')[-1]
GGML_VERSION = "gguf"

# Install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!pip install -r llama.cpp/requirements.txt

In [ ]:
# Download model
!git lfs install
!git clone https://huggingface.co/{MODEL_ID}

In [ ]:

# Convert to fp16
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{GGML_VERSION}.fp16.bin"
!python llama.cpp/convert.py {MODEL_NAME} --outtype f16 --outfile {fp16}

# Quantize the model for each method in the QUANTIZATION_METHODS list
for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{GGML_VERSION}.{method}.bin"
    !./llama.cpp/quantize {fp16} {qtype} {method}

## 추론 실행

다음은 양자화된 모델을 실행하는 간단한 스크립트입니다.추론 속도를 높이기 위해 모든 레이어를 GPU로 오프로드(7b 매개 변수 모델의 경우 35개)하고 있습니다.

In [ ]:
import os

model_list = [file for file in os.listdir(MODEL_NAME) if GGML_VERSION in file]

prompt = input("Enter your prompt: ")
chosen_method = input("Please specify the quantization method to run the model (options: " + ", ".join(model_list) + "): ")

# Verify the chosen method is in the list
if chosen_method not in model_list:
    print("Invalid method chosen!")
else:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{GGML_VERSION}.{method}.bin"
    !./llama.cpp/main -m {qtype} -n 128 --color -ngl 35 -p "{prompt}"

## 허깅페이스 업로드

모델을 허깅페이스로 업로드하려면 다음을 실행합니다.그러면 "-GGUF" 접미사가 붙은 새 리포지토리가 생성됩니다.다음 블록에서 `username` 변수를 변경하는 것을 잊지 마세요.

In [ ]:
!pip install -q huggingface_hub

username = "Redwood0"

from huggingface_hub import notebook_login, create_repo, HfApi
notebook_login()

In [ ]:
api = HfApi()


create_repo(repo_id = f"{username}/{MODEL_NAME}-GGUF", repo_type="model", exist_ok=True)

api.upload_folder(
    folder_path=MODEL_NAME,
    repo_id=f"{username}/{MODEL_NAME}-GGUF",
    allow_patterns=f"*{GGML_VERSION}*",
)